In [1]:
import pandas as pd
import numpy as np
import ast

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

C:\Users\fabio\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
'''
annotated_df = pd.read_csv("data/annotated_dataset/annotated_texts_repr_pro_complete.csv", sep=",", encoding="utf-8")
meta_df = pd.read_csv("data/cleaned_dataset/meta.tsv", sep=",", encoding="utf-8")
merged_df = annotated_df.merge(meta_df, how='left', left_on='id', right_on="ID")

ling_prof_df = merged_df[["id", "pop_sum","polarization","Speaker_ID", "Speaker_party","Party_orientation","linguistic_profile_pro", "tfidf_pro", "doc_embedding_pro"]]
diz = {
    "LN-Aut": "Lega",
    "L-SP": "Lega",
    "M5S.1": "M5S",
    "M5S.2": "M5S",
}
ling_prof_df = ling_prof_df.replace({"Speaker_party": diz})
ling_prof_df = ling_prof_df.rename(columns={"linguistic_profile_pro": "linguistic_profile", "tfidf_pro": "tfidf", "doc_embedding_pro":"doc_embedding"})
'''

In [3]:
'''
def mean_std_lists(lists):
    lists = np.array([ast.literal_eval(item) for item in lists])
    return np.mean(lists, axis=0).tolist(), np.std(lists, axis=0).tolist()

#Aggregazione per senatore
df_grouped = ling_prof_df.groupby('Speaker_ID', as_index=False).agg({
    'pop_sum': lambda x: x.sum() / x.count(),
    'polarization': lambda x: x.sum() / x.count(),
    'Speaker_party': lambda x: x.mode()[0],
    'Party_orientation': lambda x: x.mode()[0]
})

#Media e std delle rappresentazioni del testo (per tutti i testi di un senatore)
for col in ['linguistic_profile', 'tfidf', 'doc_embedding']:
    df_grouped[col], df_grouped[col + '_std'] = zip(*ling_prof_df.groupby('Speaker_ID')[col].apply(lambda x: mean_std_lists(list(x))))

df_grouped["linguistic_profile"] = df_grouped["linguistic_profile"] + df_grouped["linguistic_profile_std"]
df_grouped["tfidf"] = df_grouped["tfidf"] + df_grouped["tfidf_std"]
df_grouped["doc_embedding"] = df_grouped["doc_embedding"] + df_grouped["doc_embedding_std"]
    
group_sizes = ling_prof_df.groupby('Speaker_ID').size().reset_index(name='Count_Per_Group')
df_grouped = df_grouped.merge(group_sizes, on='Speaker_ID')
'''

In [5]:
#df_grouped.to_csv("data/annotated_dataset/speaker_data.csv", index=False)

In [3]:
scaler = StandardScaler()
def linearSvcBestParams(reprs, y):
    for name, X in reprs.items():
        print(name)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        svc = LinearSVC(dual=False, max_iter=10000)

        param_grid = {
            'C': [0.01, 0.1, 1],
            'loss': ['squared_hinge', 'hinge'],
            'penalty': ['l1', 'l2'],
            'class_weight': ['balanced']
        }

        grid_search = GridSearchCV(svc, param_grid, cv=2, scoring='accuracy', n_jobs=-1, verbose=2)
        grid_search.fit(X_train, y_train)

        best_params = grid_search.best_params_
        print("Best Parameters:", best_params)
        print("Best Cross-Validation Accuracy:", grid_search.best_score_)

        best_model = LinearSVC(dual=False, **best_params)

        X_scaled = scaler.transform(X)
        predictions = cross_val_predict(best_model, X_scaled, y, cv=3)

        print(classification_report(y, predictions))

#### Estraggo X e y

In [4]:
speaker_df = pd.read_csv("data/annotated_dataset/speaker_data.csv", sep=",", encoding="utf-8")
speaker_df = speaker_df[speaker_df["Count_Per_Group"] > 10]

x_tfidf = speaker_df["tfidf"]
x_tfidf = np.array([ast.literal_eval(item) for item in x_tfidf])

x_docembedding = speaker_df["doc_embedding"]
x_docembedding = np.array([ast.literal_eval(item) for item in x_docembedding])

x_pos_docembedding = speaker_df["pos_docembedding"]
x_pos_docembedding = np.array([ast.literal_eval(item) for item in x_pos_docembedding])

x_linguistic_profile = speaker_df["linguistic_profile"]
x_linguistic_profile = np.array([ast.literal_eval(item) for item in x_linguistic_profile])

reprs = {
    "tfidf": x_tfidf,
    "doc_embedding": x_docembedding,
    "pos_docembedding": x_pos_docembedding,
    "linguistic_profile": x_linguistic_profile
}

y_party = speaker_df["Speaker_party"]
y_ori = speaker_df["Party_orientation"]

In [5]:
X_ling = np.vstack(df_grouped["linguistic_profile"].values)
X_tfidf = np.vstack(df_grouped["tfidf"].values)
X_embed = np.vstack(df_grouped["doc_embedding"].values)

X_list = [X_ling,X_tfidf,X_embed]

y_party = np.vstack(df_grouped["Speaker_party"].values)
y_ori = np.vstack(df_grouped["Party_orientation"].values)

## Classificazione

In [5]:
clf_linear_svc = LinearSVC(C=1.0, random_state=42)

In [6]:
def classificazione(reprs, y):
    for name, X in reprs.items():
        print(name)
        predictions = cross_val_predict(clf_linear_svc, X, y, cv=3)
        print(classification_report(y, predictions))

In [7]:
classificazione(reprs, y_party)

tfidf
              precision    recall  f1-score   support

       FI-BP       0.71      0.57      0.63        30
         FdI       0.46      0.35      0.40        17
        Lega       0.41      0.33      0.37        39
         M5S       0.61      0.73      0.66        73
         NCD       0.12      0.06      0.08        17
          PD       0.61      0.73      0.67       108
         PdL       0.42      0.33      0.37        24
        SCpI       0.33      0.14      0.20         7

    accuracy                           0.57       315
   macro avg       0.46      0.41      0.42       315
weighted avg       0.54      0.57      0.55       315

doc_embedding
              precision    recall  f1-score   support

       FI-BP       0.70      0.53      0.60        30
         FdI       0.46      0.35      0.40        17
        Lega       0.33      0.31      0.32        39
         M5S       0.58      0.66      0.62        73
         NCD       0.10      0.06      0.07        17
    

In [8]:
classificazione(reprs, y_ori)

tfidf
                             precision    recall  f1-score   support

                     Centro       0.00      0.00      0.00         7
 Centro verso centro-destra       0.11      0.06      0.08        17
              Centro-destra       0.63      0.59      0.61        54
            Centro-sinistra       0.64      0.69      0.67       108
Destra verso estrema destra       0.51      0.54      0.52        56
                Pigliatutto       0.66      0.70      0.68        73

                   accuracy                           0.60       315
                  macro avg       0.43      0.43      0.43       315
               weighted avg       0.58      0.60      0.59       315

doc_embedding
                             precision    recall  f1-score   support

                     Centro       0.00      0.00      0.00         7
 Centro verso centro-destra       0.12      0.06      0.08        17
              Centro-destra       0.60      0.54      0.57        54
          

In [9]:
linearSvcBestParams(reprs, y_party)

tfidf
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.7103174603174603
              precision    recall  f1-score   support

       FI-BP       0.68      0.83      0.75        30
         FdI       0.87      0.76      0.81        17
        Lega       0.86      0.92      0.89        39
         M5S       0.85      0.92      0.88        73
         NCD       0.45      0.29      0.36        17
          PD       0.79      0.80      0.79       108
         PdL       0.71      0.50      0.59        24
        SCpI       0.20      0.14      0.17         7

    accuracy                           0.78       315
   macro avg       0.67      0.65      0.65       315
weighted avg       0.77      0.78      0.77       315

doc_embedding
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.01, 'class_weight': 'balanced', '

In [10]:
linearSvcBestParams(reprs, y_ori)

tfidf
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.7301587301587302
                             precision    recall  f1-score   support

                     Centro       0.20      0.14      0.17         7
 Centro verso centro-destra       0.38      0.29      0.33        17
              Centro-destra       0.74      0.78      0.76        54
            Centro-sinistra       0.79      0.79      0.79       108
Destra verso estrema destra       0.89      0.86      0.87        56
                Pigliatutto       0.87      0.93      0.90        73

                   accuracy                           0.79       315
                  macro avg       0.64      0.63      0.64       315
               weighted avg       0.78      0.79      0.79       315

doc_embedding
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters:

### Selezione di soli due partiti

In [11]:
filtered_df = speaker_df[speaker_df["Speaker_party"].isin(["PD", "M5S"])]

x_tfidf = np.array([ast.literal_eval(item) for item in filtered_df["tfidf"]])
x_docembedding = np.array([ast.literal_eval(item) for item in filtered_df["doc_embedding"]])
x_linguistic_profile = np.array([ast.literal_eval(item) for item in filtered_df["linguistic_profile"]])

reprs = {
    "tfidf": x_tfidf,
    "doc_embedding": x_docembedding,
    "linguistic_profile": x_linguistic_profile
}

y_party = filtered_df["Speaker_party"].values
y_ori = filtered_df["Party_orientation"].values

In [12]:
classificazione(reprs, y_party)

tfidf
              precision    recall  f1-score   support

         M5S       0.77      0.82      0.79        73
          PD       0.87      0.83      0.85       108

    accuracy                           0.83       181
   macro avg       0.82      0.83      0.82       181
weighted avg       0.83      0.83      0.83       181

doc_embedding
              precision    recall  f1-score   support

         M5S       0.73      0.75      0.74        73
          PD       0.83      0.81      0.82       108

    accuracy                           0.79       181
   macro avg       0.78      0.78      0.78       181
weighted avg       0.79      0.79      0.79       181

linguistic_profile
              precision    recall  f1-score   support

         M5S       0.48      0.93      0.64        73
          PD       0.88      0.32      0.47       108

    accuracy                           0.57       181
   macro avg       0.68      0.63      0.55       181
weighted avg       0.72      0.57  

In [13]:
classificazione(reprs, y_ori)

tfidf
                 precision    recall  f1-score   support

Centro-sinistra       0.87      0.83      0.85       108
    Pigliatutto       0.77      0.82      0.79        73

       accuracy                           0.83       181
      macro avg       0.82      0.83      0.82       181
   weighted avg       0.83      0.83      0.83       181

doc_embedding
                 precision    recall  f1-score   support

Centro-sinistra       0.83      0.81      0.82       108
    Pigliatutto       0.73      0.75      0.74        73

       accuracy                           0.79       181
      macro avg       0.78      0.78      0.78       181
   weighted avg       0.79      0.79      0.79       181

linguistic_profile
                 precision    recall  f1-score   support

Centro-sinistra       0.75      0.67      0.71       108
    Pigliatutto       0.58      0.67      0.62        73

       accuracy                           0.67       181
      macro avg       0.66      0.67     

In [14]:
linearSvcBestParams(reprs, y_party)

tfidf
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.8611111111111112
              precision    recall  f1-score   support

         M5S       0.80      0.95      0.87        73
          PD       0.96      0.84      0.90       108

    accuracy                           0.88       181
   macro avg       0.88      0.89      0.88       181
weighted avg       0.90      0.88      0.89       181

doc_embedding
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.01, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l2'}
Best Cross-Validation Accuracy: 0.8402777777777778
              precision    recall  f1-score   support

         M5S       0.76      0.86      0.81        73
          PD       0.90      0.81      0.85       108

    accuracy                           0.83       181
   macro avg  

In [15]:
linearSvcBestParams(reprs, y_ori)

tfidf
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.8611111111111112
                 precision    recall  f1-score   support

Centro-sinistra       0.96      0.84      0.90       108
    Pigliatutto       0.80      0.95      0.87        73

       accuracy                           0.88       181
      macro avg       0.88      0.89      0.88       181
   weighted avg       0.90      0.88      0.89       181

doc_embedding
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.01, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l2'}
Best Cross-Validation Accuracy: 0.8402777777777778
                 precision    recall  f1-score   support

Centro-sinistra       0.90      0.81      0.85       108
    Pigliatutto       0.76      0.86      0.81        73

       accuracy                          

### Selezione di soli quattro partiti

In [16]:
filtered_df = speaker_df[speaker_df["Speaker_party"].isin(["PD", "M5S", "Lega", "FdI"])]

x_tfidf = np.array([ast.literal_eval(item) for item in filtered_df["tfidf"]])
x_docembedding = np.array([ast.literal_eval(item) for item in filtered_df["doc_embedding"]])
x_linguistic_profile = np.array([ast.literal_eval(item) for item in filtered_df["linguistic_profile"]])

reprs = {
    "tfidf": x_tfidf,
    "doc_embedding": x_docembedding,
    "linguistic_profile": x_linguistic_profile
}

y_party = filtered_df["Speaker_party"].values
y_ori = filtered_df["Party_orientation"].values

In [17]:
classificazione(reprs, y_party)

tfidf
              precision    recall  f1-score   support

         FdI       0.64      0.41      0.50        17
        Lega       0.52      0.41      0.46        39
         M5S       0.68      0.70      0.69        73
          PD       0.72      0.81      0.76       108

    accuracy                           0.68       237
   macro avg       0.64      0.58      0.60       237
weighted avg       0.67      0.68      0.67       237

doc_embedding
              precision    recall  f1-score   support

         FdI       0.60      0.35      0.44        17
        Lega       0.41      0.33      0.37        39
         M5S       0.64      0.68      0.66        73
          PD       0.72      0.78      0.75       108

    accuracy                           0.65       237
   macro avg       0.59      0.54      0.55       237
weighted avg       0.63      0.65      0.64       237

linguistic_profile
              precision    recall  f1-score   support

         FdI       0.33      0.12   

In [18]:
classificazione(reprs, y_ori)

tfidf
                             precision    recall  f1-score   support

            Centro-sinistra       0.73      0.79      0.76       108
Destra verso estrema destra       0.63      0.55      0.59        56
                Pigliatutto       0.68      0.67      0.68        73

                   accuracy                           0.70       237
                  macro avg       0.68      0.67      0.68       237
               weighted avg       0.69      0.70      0.69       237

doc_embedding
                             precision    recall  f1-score   support

            Centro-sinistra       0.72      0.77      0.74       108
Destra verso estrema destra       0.57      0.54      0.55        56
                Pigliatutto       0.65      0.60      0.62        73

                   accuracy                           0.66       237
                  macro avg       0.64      0.64      0.64       237
               weighted avg       0.66      0.66      0.66       237

linguist

In [19]:
linearSvcBestParams(reprs, y_party)

tfidf
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.8678611422172452
              precision    recall  f1-score   support

         FdI       0.94      0.94      0.94        17
        Lega       0.89      0.85      0.87        39
         M5S       0.88      0.93      0.91        73
          PD       0.93      0.92      0.93       108

    accuracy                           0.91       237
   macro avg       0.91      0.91      0.91       237
weighted avg       0.91      0.91      0.91       237

doc_embedding
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.6667413213885778
              precision    recall  f1-score   support

         FdI       0.50      0.53      0.51        17
        Lega       0

In [20]:
linearSvcBestParams(reprs, y_ori)

tfidf
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.8889137737961925
                             precision    recall  f1-score   support

            Centro-sinistra       0.92      0.91      0.91       108
Destra verso estrema destra       0.91      0.91      0.91        56
                Pigliatutto       0.91      0.92      0.91        73

                   accuracy                           0.91       237
                  macro avg       0.91      0.91      0.91       237
               weighted avg       0.91      0.91      0.91       237

doc_embedding
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l2'}
Best Cross-Validation Accuracy: 0.7298432250839866
                             precision    recall  f1-score   support

      

### Selezione di partiti con support >47

In [21]:
filtered_df = speaker_df[speaker_df["Speaker_party"].isin(["PD", "M5S", "Lega", "PdL", "FI-BP"])]

x_tfidf = np.array([ast.literal_eval(item) for item in filtered_df["tfidf"]])
x_docembedding = np.array([ast.literal_eval(item) for item in filtered_df["doc_embedding"]])
x_linguistic_profile = np.array([ast.literal_eval(item) for item in filtered_df["linguistic_profile"]])

reprs = {
    "tfidf": x_tfidf,
    "doc_embedding": x_docembedding,
    "linguistic_profile": x_linguistic_profile
}

y_party = filtered_df["Speaker_party"].values
y_ori = filtered_df["Party_orientation"].values

In [22]:
classificazione(reprs, y_party)

tfidf
              precision    recall  f1-score   support

       FI-BP       0.74      0.67      0.70        30
        Lega       0.45      0.46      0.46        39
         M5S       0.65      0.67      0.66        73
          PD       0.70      0.80      0.74       108
         PdL       0.67      0.25      0.36        24

    accuracy                           0.65       274
   macro avg       0.64      0.57      0.59       274
weighted avg       0.65      0.65      0.64       274

doc_embedding
              precision    recall  f1-score   support

       FI-BP       0.68      0.63      0.66        30
        Lega       0.35      0.33      0.34        39
         M5S       0.59      0.64      0.62        73
          PD       0.68      0.77      0.72       108
         PdL       0.50      0.17      0.25        24

    accuracy                           0.61       274
   macro avg       0.56      0.51      0.52       274
weighted avg       0.59      0.61      0.59       274

li

In [23]:
classificazione(reprs, y_ori)

tfidf
                             precision    recall  f1-score   support

              Centro-destra       0.68      0.59      0.63        54
            Centro-sinistra       0.71      0.81      0.75       108
Destra verso estrema destra       0.50      0.44      0.47        39
                Pigliatutto       0.71      0.68      0.70        73

                   accuracy                           0.68       274
                  macro avg       0.65      0.63      0.64       274
               weighted avg       0.67      0.68      0.67       274

doc_embedding
                             precision    recall  f1-score   support

              Centro-destra       0.62      0.56      0.59        54
            Centro-sinistra       0.69      0.77      0.72       108
Destra verso estrema destra       0.42      0.33      0.37        39
                Pigliatutto       0.64      0.64      0.64        73

                   accuracy                           0.63       274
         

In [24]:
linearSvcBestParams(reprs, y_party)

tfidf
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.7989157631359466
              precision    recall  f1-score   support

       FI-BP       0.72      0.70      0.71        30
        Lega       0.88      0.90      0.89        39
         M5S       0.91      0.92      0.91        73
          PD       0.85      0.91      0.88       108
         PdL       0.69      0.46      0.55        24

    accuracy                           0.85       274
   macro avg       0.81      0.78      0.79       274
weighted avg       0.84      0.85      0.84       274

doc_embedding
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.5754378648874061
              precision    recall  f1-score   support

       FI-BP      

In [25]:
linearSvcBestParams(reprs, y_ori)

tfidf
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.8127189324437031
                             precision    recall  f1-score   support

              Centro-destra       0.92      0.81      0.86        54
            Centro-sinistra       0.88      0.92      0.90       108
Destra verso estrema destra       0.93      0.95      0.94        39
                Pigliatutto       0.93      0.93      0.93        73

                   accuracy                           0.91       274
                  macro avg       0.91      0.90      0.91       274
               weighted avg       0.91      0.91      0.90       274

doc_embedding
Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge', 'penalty': 'l1'}
Best Cross-Validation Accuracy: 0.6211009174311927
     